In [2]:
%load_ext autoreload

%cd /home/nicolo_b/Desktop/PhD/IFX/Notebooks/deepLearning/TRANSFER_LEARNING_JOURNAL_PAPER
import sys,os
sys.path.append('../')
import nn_models
from nn_models import *

import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from parse import *


from sklearn.linear_model import RidgeCV

from sklearn.linear_model import LinearRegression
import torch
from torch import nn
import pytorch_lightning as pl
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import os


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


from datetime import datetime
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingRegressor


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/nicolo_b/Desktop/PhD/IFX/Notebooks/deepLearning/TRANSFER_LEARNING_JOURNAL_PAPER


In [3]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
from sklearn.datasets import make_regression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
X, _  = make_regression(n_samples=3000, n_features=27, effective_rank = 14, n_informative=27, noise=0.1, random_state=SEED)

# add non_linearity to mimic polynomial features interactions
ground_truth = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(2)),
    ('pca', PCA(1,random_state=SEED)),
])

y = ground_truth.fit_transform(X).reshape(-1)
y = MinMaxScaler().fit_transform(y.reshape(-1,1)).reshape(-1)



In [5]:
checkpoint_path = f'./checkpoints'
checkpoint_name_fe = 'SoftOrdering_PL_FE.ckpt'
checkpoint_name_fe_ae = 'SoftOrdering_AE_FE.ckpt'

In [6]:
parameter_dict = {'input_dim':27,
        'sign_size':32,
        'cha_input':32,
        'cha_hidden':32,
        'K':2,
        'dropout_input':0.1,
        'dropout_hidden':0.1,
        'dropout_output':0.1}

parameter_dict_ae = {'input_dim':27,
        'sign_size':32,
        'cha_input':32,
        'cha_hidden':64,
        'K':4,
        'dropout_input':0.2,
        'dropout_hidden':0.2,
        'dropout_output':0.2}

In [7]:
BEST_COLUMNS = np.arange(27)
COLUMNS = np.arange(27)
def create_reg():
    regressors_dict = {
        'Baseline': Pipeline([
            ('scaler',StandardScaler()),
            ('PCA',PCA(14)),
            ('transformer',PolynomialFeatures(2)),
            ('regressor',RidgeCV(alphas=np.logspace(-6,6,2000)))
            ]),
        'Dummy Voting CNN': VotingRegressor(
            [(f'Dummy CNN {i}',Pipeline([
                ('scaler',StandardScaler()),
                ('selector',DummySelector(columns=COLUMNS,features_len=27)),
                ('transformer',SoftOrderingTrasnformer(SoftOrdering1DCNN,parameter_dict,allow_training=False,pre_trained_ckpt=os.path.join(checkpoint_path,checkpoint_name_fe),callbacks = 
                 [EarlyStopping(monitor='valid_loss',min_delta=.01,patience=30,verbose=True,mode='min')])),                
                ('regressor',RidgeCV(alphas=np.logspace(-6,6,2000)))])) for i in range(10)],n_jobs=1),
        
        'SoftOrderingCNN' : Pipeline([
        ('scaler',StandardScaler()),
        ('transformer',SoftOrderingTrasnformer(SoftOrdering1DCNN,parameter_dict,pre_trained_ckpt=os.path.join(checkpoint_path,checkpoint_name_fe),
                                               device='cuda',allow_training=False)),
        ('regressor',RidgeCV(alphas=np.logspace(-6,6,2000)))
        ]),
        
         'SoftOrderingCNN_AE' : Pipeline([
        ('scaler',StandardScaler()),
        ('transformer',SoftOrderingTrasnformer(SoftOrdering1DCNN_AutoEncoder,parameter_dict_ae,pre_trained_ckpt=os.path.join(checkpoint_path,checkpoint_name_fe_ae),
                                               device='cuda',allow_training=False)),
        ('regressor',RidgeCV(alphas=np.logspace(-6,6,2000)))
        ]),         
        'KNN-5': Pipeline([
            ('scaler',StandardScaler()),
            ('regressor',KNeighborsRegressor(n_neighbors=5,n_jobs=-1))
            ]),
        'RandomForest':
            Pipeline([
            ('scaler',StandardScaler()),
            ('regressor',RandomForestRegressor(n_estimators=100,n_jobs=-1))
            ]),
        'Linear Ridge': Pipeline([
            ('scaler',StandardScaler()),
            ('regressor',RidgeCV(alphas=np.logspace(-6,6,2000)))
          ]),
    
        'Linear Reg': Pipeline([
            ('scaler',StandardScaler()),
            ('regressor',LinearRegression())
          ])

        }
    return regressors_dict

In [8]:
from collections import defaultdict
mean_scores = defaultdict(list)
regressors_dict = create_reg()
for name,reg in regressors_dict.items():
    for i in range(5):
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=i)
        reg.fit(X_train,y_train)
        y_prd = reg.predict(X_test)
        mean_scores[name].append(r2_score(y_test,y_prd))
        print(f'{name} {mean_scores[name][-1]}')
    print(f'{name} mean {np.mean(mean_scores[name])}')

Baseline 0.9723106997260786
Baseline 0.9739899725241814
Baseline 0.976971479897263
Baseline 0.9728230090756222
Baseline 0.9743373121361071
Baseline mean 0.9740864946718505
Dummy Voting CNN 0.7660120174640297
Dummy Voting CNN 0.7448891802064432
Dummy Voting CNN 0.7822704776095264
Dummy Voting CNN 0.7881667451116458
Dummy Voting CNN 0.7902418416967
Dummy Voting CNN mean 0.7743160524176689
SoftOrderingCNN 0.5570689769315814
SoftOrderingCNN 0.617695202279043
SoftOrderingCNN 0.5998603592328945
SoftOrderingCNN 0.6222708720891356
SoftOrderingCNN 0.5711633917245278
SoftOrderingCNN mean 0.5936117604514364
SoftOrderingCNN_AE 0.1711721786725039
SoftOrderingCNN_AE 0.1603010102346355
SoftOrderingCNN_AE 0.12559652051420045
SoftOrderingCNN_AE 0.1337224707244573
SoftOrderingCNN_AE 0.08998018596003765
SoftOrderingCNN_AE mean 0.13615447322116697
KNN-5 0.3888188182775054
KNN-5 0.3722586147532636
KNN-5 0.3657186460132896
KNN-5 0.36125980603650387
KNN-5 0.40593483132124053
KNN-5 mean 0.37879814328036054
Ra